# Rf 2 0 2_Extendedmlfit
Addition and convolution: setting up an extended maximum likelihood fit



**Author:** 07/2008 - Wouter Verkerke  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Thursday, August 29, 2019 at 02:43 AM.</small></i>

In [1]:
%%cpp -d
#include "RooRealVar.h"
#include "RooDataSet.h"
#include "RooGaussian.h"
#include "RooChebychev.h"
#include "RooAddPdf.h"
#include "RooExtendPdf.h"
#include "TCanvas.h"
#include "TAxis.h"
#include "RooPlot.h"

In [2]:
%%cpp -d
// This is a workaround to make sure the namespace is used inside functions
using namespace RooFit;

Setup component pdfs
 ---------------------------------------

Declare observable x

In [3]:
RooRealVar x("x", "x", 0, 10);


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



Create two gaussian pdfs g1(x,mean1,sigma) anf g2(x,mean2,sigma) and their parameters

In [4]:
RooRealVar mean("mean", "mean of gaussians", 5);
RooRealVar sigma1("sigma1", "width of gaussians", 0.5);
RooRealVar sigma2("sigma2", "width of gaussians", 1);

RooGaussian sig1("sig1", "Signal component 1", x, mean, sigma1);
RooGaussian sig2("sig2", "Signal component 2", x, mean, sigma2);

Build chebychev polynomial p.d.f.

In [5]:
RooRealVar a0("a0", "a0", 0.5, 0., 1.);
RooRealVar a1("a1", "a1", 0.2, 0., 1.);
RooChebychev bkg("bkg", "Background", x, RooArgSet(a0, a1));

Sum the signal components into a composite signal p.d.f.

In [6]:
RooRealVar sig1frac("sig1frac", "fraction of component 1 in signal", 0.8, 0., 1.);
RooAddPdf sig("sig", "Signal", RooArgList(sig1, sig2), sig1frac);

----------------
METHOD 1
================

Construct extended composite model
 -------------------------------------------------------------------

Sum the composite signal and background into an extended pdf nsig*sig+nbkg*bkg

In [7]:
RooRealVar nsig("nsig", "number of signal events", 500, 0., 10000);
RooRealVar nbkg("nbkg", "number of background events", 500, 0, 10000);
RooAddPdf model("model", "(g1+g2)+a", RooArgList(bkg, sig), RooArgList(nbkg, nsig));

Sample, fit and plot extended model
 ---------------------------------------------------------------------

Generate a data sample of expected number events in x from model
 = model.expectedEvents() = nsig+nbkg

In [8]:
RooDataSet *data = model.generate(x);

Fit model to data, extended ml term automatically included

In [9]:
model.fitTo(*data);

[#1] INFO:Minization -- p.d.f. provides expected number of events, including extended term in likelihood.
[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
[#1] INFO:Minization --  The following expressions have been identified as constant and will be precalculated and cached: (sig1,sig2)
[#1] INFO:Minization --  The following expressions will be evaluated in cache-and-track mode: (bkg)
 **********
 **    1 **SET PRINT           1
 **********
 **********
 **    2 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 a0           5.00000e-01  1.00000e-01    0.00000e+00  1.00000e+00
     2 a1           2.00000e-01  1.00000e-01    0.00000e+00  1.00000e+00
     3 nbkg         5.00000e+02  2.50000e+02    0.00000e+00  1.00000e+04
     4 nsig         5.00000e+02  2.50000e+02    0.00000e+00  1.00000e+04
     5 sig1frac     8.00000e-01  1.00000e-01    0.00000e+00  1.00000e+00
 **********
 **    3 **SET ER

Plot data and pdf overlaid, use expected number of events for p.d.f projection normalization
 rather than observed number of events (==data->numEntries())

In [10]:
RooPlot *xframe = x.frame(Title("extended ML fit example"));
data->plotOn(xframe);
model.plotOn(xframe, Normalization(1.0, RooAbsReal::RelativeExpected));

Overlay the background component of model with a dashed line

In [11]:
model.plotOn(xframe, Components(bkg), LineStyle(kDashed), Normalization(1.0, RooAbsReal::RelativeExpected));

[#1] INFO:Plotting -- RooAbsPdf::plotOn(model) directly selected PDF components: (bkg)
[#1] INFO:Plotting -- RooAbsPdf::plotOn(model) indirectly selected PDF components: ()


Overlay the background+sig2 components of model with a dotted line

In [12]:
model.plotOn(xframe, Components(RooArgSet(bkg, sig2)), LineStyle(kDotted),
             Normalization(1.0, RooAbsReal::RelativeExpected));

[#1] INFO:Plotting -- RooAbsPdf::plotOn(model) directly selected PDF components: (bkg,sig2)
[#1] INFO:Plotting -- RooAbsPdf::plotOn(model) indirectly selected PDF components: (sig)


Print structure of composite p.d.f.

In [13]:
model.Print("t");

0x7f39b5d82750 RooAddPdf::model = 0.898614 [Auto,Dirty] 
  0x7f39b702cb08/V- RooChebychev::bkg = 0.798919 [Auto,Dirty] 
    0x7f39b702c010/V- RooRealVar::x = 5
    0x7f39b702c3b8/V- RooRealVar::a0 = 0.441701 +/- 0.073185
    0x7f39b702c760/V- RooRealVar::a1 = 0.201081 +/- 0.117613
  0x7f39b5d823a8/V- RooRealVar::nbkg = 504.206 +/- 39.3107
  0x7f39b5d8a870/V- RooAddPdf::sig = 1 [Auto,Dirty] 
    0x7f39b5d89af8/V- RooGaussian::sig1 = 1 [Auto,Dirty] 
      0x7f39b702c010/V- RooRealVar::x = 5
      0x7f39b5d89000/V- RooRealVar::mean = 5
      0x7f39b5d893a8/V- RooRealVar::sigma1 = 0.5
    0x7f39b5d8a4c8/V- RooRealVar::sig1frac = 0.837341 +/- 0.116841
    0x7f39b5d89fe0/V- RooGaussian::sig2 = 1 [Auto,Dirty] 
      0x7f39b702c010/V- RooRealVar::x = 5
      0x7f39b5d89000/V- RooRealVar::mean = 5
      0x7f39b5d89750/V- RooRealVar::sigma2 = 1
  0x7f39b5d82000/V- RooRealVar::nsig = 495.799 +/- 39.2054


----------------
METHOD 2
================

Construct extended components first
 ---------------------------------------------------------------------

Associated nsig/nbkg as expected number of events with sig/bkg

In [14]:
RooExtendPdf esig("esig", "extended signal p.d.f", sig, nsig);
RooExtendPdf ebkg("ebkg", "extended background p.d.f", bkg, nbkg);

Sum extended components without coefs
 -------------------------------------------------------------------------

Construct sum of two extended p.d.f. (no coefficients required)

In [15]:
RooAddPdf model2("model2", "(g1+g2)+a", RooArgList(ebkg, esig));

Draw the frame on the canvas

In [16]:
new TCanvas("rf202_composite", "rf202_composite", 600, 600);
gPad->SetLeftMargin(0.15);
xframe->GetYaxis()->SetTitleOffset(1.4);
xframe->Draw();

Draw all canvases 

In [17]:
%jsroot on
gROOT->GetListOfCanvases()->Draw()